In [1]:
# 필수 라이브러리 호출
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [17]:
# 데이터 정의

st_default_df = pd.read_csv('./ysp_files/st_default.csv')
st_default_df1 = st_default_df.iloc[:,1:]

,default,student,balance,income
0,No,No,729.526495,44361.62507
1,No,Yes,817.180407,12106.13470
2,No,No,1073.549164,31767.13895
3,No,No,529.250605,35704.49394
4,No,No,785.655883,38463.49588
...,...,...,...,...
9995,No,No,711.555020,52992.37891
9996,No,No,757.962918,19660.72177
9997,No,No,845.411989,58636.15698
9998,No,No,1569.009053,36669.11236


In [3]:
# train_test_split 하기 전에 문자형 데이터들 변경 - 1)레이블 vs 2) one-hot
# 그런데 y_label의 경우 default를 예측하는 값이므로 이 값은 label
# 나머지 student는 get_dummies()

X_data = st_default_df1.drop('default', axis=1)
y_target = st_default_df1['default']

In [4]:
# 크기 확인 
print('X_data의 크기:', X_data.shape[0])
print('y_target의 크기:', y_target.shape[0])

X_data의 크기: 10000
y_target의 크기: 10000


In [20]:
X_data_df = pd.get_dummies(X_data)  # pandas 자체적으로 get_dummies()를 쓰면 object 컬럼이 onehot 전처리 된것처럼 0과 1로 나누어 진다.
X_data_df

,balance,income,student_No,student_Yes
0,729.526495,44361.62507,1,0
1,817.180407,12106.13470,0,1
2,1073.549164,31767.13895,1,0
3,529.250605,35704.49394,1,0
4,785.655883,38463.49588,1,0
...,...,...,...,...
9995,711.555020,52992.37891,1,0
9996,757.962918,19660.72177,1,0
9997,845.411989,58636.15698,1,0
9998,1569.009053,36669.11236,1,0


In [6]:
# y_값에 label encoding

from sklearn.preprocessing import LabelEncoder

le_encode = LabelEncoder()
y_target_scaled = le_encode.fit_transform(y_target)

# imbalanced data
pd.Series(y_target_scaled).value_counts()

0    9667
1     333
dtype: int64

In [7]:
# 데이터 분할 - train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data_df, y_target_scaled, stratify=y_target_scaled)

In [8]:
# 의사결정나무 적용

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train,y_train)

y_pred = dt_clf.predict(X_test)

In [9]:
# 평가지표

print(f'정확도는 :{accuracy_score(y_pred,y_test)}') 

정확도는 :0.956


#### 배우신 분들을 위한 내용

In [10]:
# 1. 교차검증을 수행
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, X_data_df, y_target_scaled, scoring='accuracy', cv=10)
print(f'교차 검증별 정확도:{np.round(scores, 4)}')
print(f'평균 검증별 정확도:{np.round(np.mean(scores), 4)}')

교차 검증별 정확도:[0.952 0.96  0.949 0.955 0.954 0.957 0.956 0.958 0.956 0.952]
평균 검증별 정확도:0.9549


In [11]:
# 2. 하이퍼파라미터 튜닝을 수행
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[3,4,5,6,7,10],
              'min_samples_split':[2,3,5,10],}
grid_dtree = GridSearchCV(dt_clf, param_grid=parameters, cv=10, refit=True)
grid_dtree.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 10],
                         'min_samples_split': [2, 3, 5, 10]})

In [12]:
estimator = grid_dtree.best_estimator_

pred_grid = estimator.predict(X_test)
print(f'test datasets accuracy:{accuracy_score(y_test,pred_grid)}')

test datasets accuracy:0.9696


In [13]:
# 1. 왜 데이터의 해당하는 정확도 값이 높게 나왔을까요?
# 2. 데이터는 어떠한 특징을 지니고 있나요?
# 3. 데이터에는 어떠한 문제점이 있나요?
# 4. 그렇다면 이러한 문제점을 해결하기 위한 방안은 무엇이 있을까요?

In [14]:
# end of file